# Results Summary

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
results_fm = pd.read_pickle("outputs/results_fm.pkl")
results_fm

In [ ]:
results_ros = pd.read_pickle("outputs/results_ros.pkl")
results_ros

In [ ]:
tab1 = results_ros.groupby('Loss', sort=False)['RMSE'].mean().reset_index()
tab1.rename(columns={'RMSE': 'Mean'})

In [ ]:
pd.options.display.float_format = '{:.2e}'.format

In [ ]:
tab1 = results_ros.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab1

In [ ]:
pd.options.display.float_format = None

In [ ]:
tab2 = results_fm.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab2.round(2)

## Plots

In [ ]:
plt.scatter(tab1.Loss, tab1.Mean, color='r')

# plt.errorbar(tab1.Loss, tab1.Mean, yerr=[tab1.Mean[i] - tab1.Min[i] for i in range(len(tab1.Loss))], fmt='o', label='Min')
# plt.errorbar(tab1.Loss, tab1.Mean, yerr=[tab1.Mean[i] - tab1.Max[i] for i in range(len(tab1.Loss))], fmt='o', label='Min')

plt.xlabel('Loss')
plt.ylabel('Mean RMSE')
plt.title('Scatter Plot of Mean RMSE by Loss Function')
plt.tick_params(axis='x', rotation=90)
plt.show()

In [ ]:
# grouped = results_ros.groupby('Loss').agg({'RMSE': ['mean', 'sem']}).reset_index()
# grouped

grouped = results_fm.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    SE=('RMSE', 'sem')
).reset_index()
grouped2 = results_ros.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    SE=('RMSE', 'sem')
).reset_index()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
ax1 = axes[0]
ax1.errorbar(grouped.Loss, grouped.Mean, yerr=grouped.SE, fmt='o', label='SE')
# plt.errorbar(tab1.Loss, tab1.Mean, yerr=[tab1.Mean[i] - tab1.Max[i] for i in range(len(tab1.Loss))], fmt='o', label='Min')
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax1.set_xlabel('Loss')
ax1.set_ylabel('Mean RMSE on FMC')
ax1.set_title('Mean RMSE by Loss Function (FMC)')
ax1.tick_params(axis='x', rotation=90)
ax1.grid()

ax1 = axes[1]
ax1.errorbar(grouped2.Loss, grouped2.Mean, yerr=grouped2.SE, fmt='o', label='SE', color='r')
# plt.errorbar(tab1.Loss, tab1.Mean, yerr=[tab1.Mean[i] - tab1.Max[i] for i in range(len(tab1.Loss))], fmt='o', label='Min')
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax1.set_xlabel('Loss')
ax1.set_ylabel('Mean RMSE on ROS')
ax1.set_title('Mean RMSE by Loss Function (ROS)')
ax1.tick_params(axis='x', rotation=90)
ax1.grid()

plt.show()

In [ ]:
models_vec = results_ros.Model.unique()

In [ ]:
results_ros

In [ ]:
N = len(models_vec) # number of rows of subplots
fig, axes = plt.subplots(N, 2, figsize=(12, 5*N))

for i in range(0, len(models_vec)):
    mod = models_vec[i]
    temp = results_ros[results_ros.Model == mod]
    temp2 = temp.groupby('Loss', sort=False).agg(
        Mean=('RMSE', 'mean'),
        SE=('RMSE', 'sem')
    ).reset_index()
    # Access the subplot at row i, column 0
    ax1 = axes[i, 0]
    # ax1.scatter(temp2.Loss, temp2.Mean)
    ax1.errorbar(temp2.Loss, temp2.Mean, yerr=temp2.SE, fmt='o', label='SE')
    ax1.tick_params(axis='x', rotation=90)
    ax1.set_title(f'Test RMSE on FMC - Model {mod}')
    ax1.grid()

    temp = results_fm[results_fm.Model == mod]
    temp2 = temp.groupby('Loss', sort=False).agg(
        Mean=('RMSE', 'mean'),
        SE=('RMSE', 'sem')
    ).reset_index()
    # Access the subplot at row i, column 0
    ax2 = axes[i, 1]
    ax2.errorbar(temp2.Loss, temp2.Mean, yerr=temp2.SE, fmt='o', label='SE', color='r')
    ax2.tick_params(axis='x', rotation=90)
    ax2.set_title(f'Test RMSE on ROS - Model {mod}')
    ax2.grid()

plt.tight_layout()
plt.show()